# Notebook to Sub file Tex

## Objective

This script is part of the hybrid chapter workflow, and to be used when you have created a ipython notebook and want to convert it to sub file tex where its preamble is removed, and instead added from the style file from our hybrid template. This sub file can be compiled separately and also becomes part of chapter, when included in the main tex file. 

## Rules
1. Try to name the notebook without any spaces (use underscore instead). 
2. Do not manually number the equations. Instead use align

## Remember
1. This only creates tex. You need to include it accordingly in main file. By default main tex has documentclass book, so recommended to add as a chapter.  

## Input Parameters

In [43]:
source_file = "../chapters/29_appendix_2.ipynb"  # file to be converted
tex_output_folder = "../chapters"
template_file = "ipy2tex_subfile_template.tplx"
temp_folder = "../tmp"
nbconvert_config_file = "nbconvert_config.py"


import ntpath
tex_file = ntpath.basename(source_file).split('.ipynb')[0] + ".tex"
tex_file_with_path_wo_ext = tex_output_folder + "/" + ntpath.basename(source_file).split('.ipynb')[0]

## Steps

1. Create the template file. This is optional once the file is created. It is here just to show whats going on in template mostly. 
2. Generate the tex file using the template file and ```nbconvert```.

> A sample notebook is used to show the demo here

### Create Template

In [44]:
%%writefile $template_file

((*- extends 'article.tplx' -*))

% 1. CHANGE DOCUMENTCLASS TO STANDALONE
((* block docclass *))
\documentclass[float=false,crop=false]{standalone}
((* endblock docclass *))




% 2. REMOVE PREAMBLE AND REPLACE WITH STYLE FILE INCLUSION
((* block packages *))
\usepackage{../myipy2tex}  % NOTE WE ARE ASSSUMING THE STYLE FILE TO BE ONE FOLDER ABOVE
\usepackage{../myipy2tex_custom}  % YOUR FURTHER CUSTOM STYLES FOR IPYTHON TO LATEX

% if you need to cross reference to any raw tex file from this resultant tex file you  need to refer them here..
% it is not needed when you compile main.tex but make sure the labels are unique
\ifstandalone
\usepackage[numbers]{natbib}
\usepackage{tikz}      % some times raw tikz needed in notebook also..
\usepackage{pgfplots}  % and so is pgfplots
\bibliographystyle{abbrvnat}
\usepackage{xr-hyper} % Needed for external references
    \externaldocument{29_appendix}
    \externaldocument{29_MLE_Regression_Main} 
\title{Hypothesis Testing}
\fi


((* endblock packages *))
((* block definitions *))
((* endblock definitions *))
((* block commands *))
((* endblock commands *))

% Draw a figure using the graphicx package. Min size due to tikzmagic outputs being too shrinked.
((* macro draw_figure(filename) -*))
((* set filename = filename | posix_path *))
((*- block figure scoped -*))
    \begin{center}
    \adjustimage{max size={0.9\linewidth}{0.9\paperheight},min size={0.5\linewidth}{!}}{((( filename )))}
    \end{center}
    { \hspace*{\fill} \\}
((*- endblock figure -*))
((*- endmacro *))


% 3. BELOW STUFF ARE JUST FOR CODE CELL CORRECTIONS TO GET LINES WRAPPED PROPERLY..
%===============================================================================
% Input
%===============================================================================

((* block input scoped *))
    ((*- if 'to_remove' in cell.metadata.get('tags', []) -*))  % remove input part of cells with tag to_remove
    %((- if cell.metadata.hide_input -))
    ((*- else -*))
        ((( custom_add_prompt(cell.source | wrap_text(88) | highlight_code(strip_verbatim=True), cell, 'In', 'incolor') )))    
    ((*- endif -*))
((* endblock input *))

%===============================================================================
% Output
%===============================================================================

% Display stream ouput with coloring
((* block stream *))
    \begin{Verbatim}[commandchars=\\\{\},fontsize=\footnotesize]
((( output.text | wrap_text(86) | escape_latex | ansi2latex )))
    \end{Verbatim}
((* endblock stream *))

%==============================================================================
% Define macro custom_add_prompt() (derived from add_prompt() macro in style_ipython.tplx)
%==============================================================================

((* macro custom_add_prompt(text, cell, prompt, prompt_color) -*))
    ((*- if cell.execution_count is defined -*))
    ((*- set execution_count = "" ~ (cell.execution_count | replace(None, " ")) -*))
    ((*- else -*))
    ((*- set execution_count = " " -*))
    ((*- endif -*))
    ((*- set indention =  " " * (execution_count | length + 7) -*))
\begin{((( prompt ~ 'Verbatim')))}[commandchars=\\\{\},fontsize=\scriptsize]
((( text | add_prompts(first='{\color{' ~ prompt_color ~ '}' ~ prompt ~ '[{\\color{' ~ prompt_color ~ '}' ~ execution_count ~ '}]:} ', cont=indention) )))
\end{((( prompt ~ 'Verbatim')))}
((*- endmacro *))

Overwriting ipy2tex_subfile_template.tplx


### Create Tex

By default, the attachments in markdown cells are not exported, so we will have a small config file to force that as well

In [45]:
%%writefile $nbconvert_config_file

c.NbConvertApp.export_format = 'latex'
c.LatexExporter.preprocessors = [
    'extract_attachments.ExtractAttachmentsPreprocessor'  # custom preprocessor from extract_attachments.py
    ]

Overwriting nbconvert_config.py


In [46]:
!jupyter nbconvert --log-level='DEBUG' --template "$template_file" "$source_file" --output-dir="$tex_output_folder" --config "$nbconvert_config_file"

[NbConvertApp] Searching ['D:\\Parthiban\\Business\\HigherStudies\\Online Courses\\Udacity\\Intro to Statistics\\29_MLE_Regression\\tools', 'C:\\Users\\parthi2929\\.jupyter', 'C:\\Users\\parthi2929\\Anaconda3\\etc\\jupyter', 'C:\\ProgramData\\jupyter'] for config files
[NbConvertApp] Looking for jupyter_config in C:\ProgramData\jupyter
[NbConvertApp] Looking for jupyter_config in C:\Users\parthi2929\Anaconda3\etc\jupyter
[NbConvertApp] Looking for jupyter_config in C:\Users\parthi2929\.jupyter
[NbConvertApp] Looking for jupyter_config in D:\Parthiban\Business\HigherStudies\Online Courses\Udacity\Intro to Statistics\29_MLE_Regression\tools
[NbConvertApp] Looking for nbconvert_config in D:\Parthiban\Business\HigherStudies\Online Courses\Udacity\Intro to Statistics\29_MLE_Regression\tools
[NbConvertApp] Loaded config file: D:\Parthiban\Business\HigherStudies\Online Courses\Udacity\Intro to Statistics\29_MLE_Regression\tools\nbconvert_config.py
[NbConvertApp] Converting notebook ../chapter

## Test pdf

This pdf is just intermediary for us to test if output is ok (should work as we have made it 'standalone' as well). This shall also go to same chapters folder for us to refer later easily. 

In [47]:
#!pdflatex.exe -synctex=1 -interaction=nonstopmode -output-directory 1 "$tex_file"
from subprocess import Popen, PIPE
process = Popen(['pdflatex.exe', '-synctex=1', '-interaction=nonstopmode', '-output-directory', tex_output_folder, '-aux-directory', temp_folder , tex_file], shell=True, stdout=PIPE)

for line in process.stdout:
    print(line.decode().strip())

This is pdfTeX, Version 3.14159265-2.6-1.40.19 (MiKTeX 2.9.6800 64-bit)
entering extended mode

("D:/Parthiban/Business/HigherStudies/Online Courses/Udacity/Intro to Statistic
s/29_MLE_Regression/chapters/29_appendix_2.tex"
LaTeX2e <2018-04-01> patch level 5
("C:\Program Files\MiKTeX 2.9\tex\latex\standalone\standalone.cls"
Document Class: standalone 2018/03/26 v1.3a Class to compile TeX sub-files stan
dalone
("C:\Program Files\MiKTeX 2.9\tex\latex\tools\shellesc.sty")
("C:\Program Files\MiKTeX 2.9\tex\generic\oberdiek\ifluatex.sty")
("C:\Program Files\MiKTeX 2.9\tex\generic\oberdiek\ifpdf.sty")
("C:\Program Files\MiKTeX 2.9\tex\generic\ifxetex\ifxetex.sty")
("C:\Program Files\MiKTeX 2.9\tex\latex\xkeyval\xkeyval.sty"
("C:\Program Files\MiKTeX 2.9\tex\generic\xkeyval\xkeyval.tex"
("C:\Program Files\MiKTeX 2.9\tex\generic\xkeyval\xkvutils.tex"
("C:\Program Files\MiKTeX 2.9\tex\generic\xkeyval\keyval.tex"))))
("C:\Program Files\MiKTeX 2.9\tex\latex\standalone\standalone.cfg")
("C:\Progra

y" ("C:\Program Files\MiKTeX 2.9\tex\latex\oberdiek\kvoptions.sty"
("C:\Program Files\MiKTeX 2.9\tex\generic\oberdiek\ltxcmds.sty")
("C:\Program Files\MiKTeX 2.9\tex\generic\oberdiek\kvsetkeys.sty"
("C:\Program Files\MiKTeX 2.9\tex\generic\oberdiek\infwarerr.sty")
("C:\Program Files\MiKTeX 2.9\tex\generic\oberdiek\etexcmds.sty")))
("C:\Program Files\MiKTeX 2.9\tex\generic\oberdiek\pdftexcmds.sty"))
("C:\Program Files\MiKTeX 2.9\tex\latex\hyperref\hyperref.sty"
("C:\Program Files\MiKTeX 2.9\tex\generic\oberdiek\hobsub-hyperref.sty"
("C:\Program Files\MiKTeX 2.9\tex\generic\oberdiek\hobsub-generic.sty"))
("C:\Program Files\MiKTeX 2.9\tex\latex\oberdiek\auxhook.sty")
("C:\Program Files\MiKTeX 2.9\tex\latex\hyperref\pd1enc.def")
("C:\Program Files\MiKTeX 2.9\tex\latex\00miktex\hyperref.cfg")
("C:\Program Files\MiKTeX 2.9\tex\latex\url\url.sty"))
("C:\Program Files\MiKTeX 2.9\tex\latex\hyperref\hpdftex.def"
("C:\Program Files\MiKTeX 2.9\tex\latex\oberdiek\rerunfilecheck.sty"))
("C:\Program 

ics/29_MLE_Regression/chapters/29_appendix_2_files/29_appendix_2_8_0.png>]
Underfull \hbox (badness 10000) in paragraph at lines 140--141

[3 <D:/Parthiban/Business/HigherStudies/Online Courses/Udacity/Intro to Statist
ics/29_MLE_Regression/chapters/29_appendix_2_files/29_appendix_2_10_0.png>]
Underfull \hbox (badness 10000) in paragraph at lines 178--179

[4 <D:/Parthiban/Business/HigherStudies/Online Courses/Udacity/Intro to Statist
ics/29_MLE_Regression/chapters/29_appendix_2_files/29_appendix_2_15_0.png>]
Underfull \hbox (badness 10000) in paragraph at lines 222--223

[5 <D:/Parthiban/Business/HigherStudies/Online Courses/Udacity/Intro to Statist
ics/29_MLE_Regression/chapters/29_appendix_2_files/29_appendix_2_18_0.png>]
Underfull \hbox (badness 10000) in paragraph at lines 270--271

[6 <D:/Parthiban/Business/HigherStudies/Online Courses/Udacity/Intro to Statist
ics/29_MLE_Regression/chapters/29_appendix_2_files/29_appendix_2_23_0.png>]

Package caption Warning: The option `hypcap=

### View pdf

In [48]:
from IPython.display import HTML, display
def PDF(url):
    return HTML('<iframe src=%s width=700 height=350></iframe>' % url)

pdf_file = "\"" + tex_output_folder + '/' + ntpath.basename(source_file).split('.ipynb')[0] + '.pdf' + "\""
print('location of test pdf: ' + pdf_file)
HTML('<iframe src={0} width=100% height=600></iframe>'.format(pdf_file))

location of test pdf: "../chapters/29_appendix_2.pdf"
